# Initialising LUSID

In [1]:
# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import os
import helper_functions as fun

# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

pd.set_option("display.precision", 8)
pd.set_option("display.float_format", "{:0.4f}".format)

print("LUSID Environment Initialised")
print(
    "LUSID API Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID API Version:  0.6.6621.0


In [2]:
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

![Scopes](img/paper-lusid.gif)

## 1) Creating a logically separated environment for each analyst

In [3]:
# Fetch our scopes
analyst_scope_code = fun.create_analyst_scope()

Analyst Scope Code: analyst-paper-396e-168b-6fe3-e8


*Our LUSID environment now looks like the below. We will update this with each step.*

![Scopes](img/paper-scopes.gif)

## 2) Creating our tradeable instrument universe for each analyst

In [4]:
# Import our instrument universe from a CSV file using Pandas
instrument_universe = pd.read_csv("data/paper-instruments.csv")
# Look at the first 10 instruments
instrument_universe.sort_values(by="instrument_name").head(n=10)

instrument_name currency          figi ticker          isin    sedol  \
0       AVEVA GRP      GBP  BBG000C21Y87    AVV  GB00BBG9VN75  BBG9VN7   
1        BAE SYS.      GBP  BBG000BD5TW4    BA.  GB0002634946   263494   
2   BATM ADVANCED      GBP  BBG000BFJD77    BVC  IL0010849045   911146   
3             BTG      GBP  BBG000BZZK79    BTG  GB0001001592   100159   
4          COBHAM      GBP  BBG000BS6810    COB  GB00B07KD360  B07KD36   
5   COMPUTACENTER      GBP  BBG000BN7CL9    CCC  GB00BV9FP302  BV9FP30   
6    CONSORT MED.      GBP  BBG000BD8278   CSRT  GB0000946276    94627   
7        DIALIGHT      GBP  BBG000BLQNG4    DIA  GB0033057794  3305779   
8           GENUS      GBP  BBG000CTMKX7    GNS  GB0002074580   207458   
9    GRESHAM TECH      GBP  BBG000BS2592    GHT  GB0008808825   880882   

  client_internal  
0    imd_56344535  
1    imd_56344983  
2    imd_56344098  
3    imd_56344345  
4    imd_56344363  
5    imd_56344296  
6    imd_56344187  
7    imd_56344572  
8    imd_56344789  
9    imd_56344132

In [5]:
batch_upsert_request = fun.batch_upsert(instrument_universe, api_factory)

Instruments Successfully Upserted: 


In [6]:
# Call LUSID to upsert our batch
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    request_body=batch_upsert_request
)

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response, identifier="Figi").sort_values(
    by="Instrument"
).head(10)

Instruments Successfully Upserted: 


Instrument       Figi ID LUSID Instrument ID
2       AVEVA GRP  BBG000C21Y87       LUID_42Q7OH5P
13       BAE SYS.  BBG000BD5TW4       LUID_9UU2Z82D
5   BATM ADVANCED  BBG000BFJD77       LUID_0XICSR6Q
29            BTG  BBG000BZZK79       LUID_VA6SLBT9
17         COBHAM  BBG000BS6810       LUID_SRP4323T
18  COMPUTACENTER  BBG000BN7CL9       LUID_0TTPI8L4
1    CONSORT MED.  BBG000BD8278       LUID_SQNT32VV
22       DIALIGHT  BBG000BLQNG4       LUID_B5KMMX6J
10          GENUS  BBG000CTMKX7       LUID_YK15OEBB
21   GRESHAM TECH  BBG000BS2592       LUID_0V8ILXJS

Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/paper-instrumentmaster.gif)

## 3) Creating a transactions portfolio

A transactions portfolio is initalised to hold all transactions made by an analyst. 

A second portfolio is also initalised which will store the index of the instrument universe to be used as a benchmark.

In [7]:
# Define unique codes for each of our portfolios
transaction_portfolio_code = "LSE_FTSE_techMarkFocus_transactions_DEMO"
reference_portfolio_code = "LSE_FTSE_techMarkFocus_reference_DEMO"
# Pretty print our codes
prettyprint.heading("Transaction Portfolio Code", transaction_portfolio_code)
prettyprint.heading("Reference Portfolio Code", reference_portfolio_code)

# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3)).isoformat()

descriptions = "Paper transaction portfolio_DEMO"
portfolio_codes = [transaction_portfolio_code, reference_portfolio_code]


fun.request_transaction_portfolio_creation(
    transaction_portfolio_code,
    portfolio_creation_date,
    analyst_scope_code,
    descriptions,
    api_factory,
)

portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3)).isoformat()

fun.request_reference_portfolio_creation(
    reference_portfolio_code, portfolio_creation_date, analyst_scope_code, api_factory
)

Transaction Portfolio Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Reference Portfolio Code: LSE_FTSE_techMarkFocus_reference_DEMO
Portfolio Created
Scope: analyst-paper-396e-168b-6fe3-e8
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Portfolio Effective From: 2021-03-20 15:01:19.034503+00:00
Portfolio Created On: 2021-03-23 15:01:19.305627+00:00

Portfolio Created
Scope: analyst-paper-396e-168b-6fe3-e8
Code: LSE_FTSE_techMarkFocus_reference_DEMO
Portfolio Effective From: 2021-03-20 15:01:19.401250+00:00
Portfolio Created On: 2021-03-23 15:01:19.617321+00:00



![Scopes](img/paper-referenceportfolio.gif)

## 4) Populating our analyst's portfolio with a starting cash balance

In [8]:
# Set the date from which the cash balance will apply to be just after portfolio creation
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=2.9)).isoformat()
# Define our initial cash balance
initial_cash_balance = 30000000

# # Create a holding adjustment to set our initial cash balance
fun.populate_with_cash(
    holdings_effective_date,
    initial_cash_balance,
    analyst_scope_code,
    transaction_portfolio_code,
    api_factory,
)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-396e-168b-6fe3-e8
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Holdings Effective From: 2021-03-20 15:01:19.034503+00:00
Holdings Created On: 2021-03-23 15:01:20.162338+00:00



Our transactions portfolio is now populated with an initial cash balance and is no longer empty.

![Scopes](img/paper-transactionsportfoliofilled.gif)

## 5) Populating our reference portfolio 

The initaised reference portfolio is populated with correct weights such that it tracks the market as closely as possible.

In [9]:
# Import the market capitalisation of each of our instruments in the index
instrument_market_cap = pd.read_csv("data/paper-weights.csv")
# Take a look at the first 10 market capitalisations
instrument_market_cap.sort_values(by="instrument_name").head(n=10)

instrument_name currency          figi ticker          isin    sedol  \
0       AVEVA GRP      GBP  BBG000C21Y87    AVV  GB00BBG9VN75  BBG9VN7   
1        BAE SYS.      GBP  BBG000BD5TW4    BA.  GB0002634946  0263494   
2   BATM ADVANCED      GBP  BBG000BFJD77    BVC  IL0010849045   911146   
3             BTG      GBP  BBG000BZZK79    BTG  GB0001001592   100159   
4          COBHAM      GBP  BBG000BS6810    COB  GB00B07KD360  B07KD36   
5   COMPUTACENTER      GBP  BBG000BN7CL9    CCC  GB00BV9FP302  BV9FP30   
6    CONSORT MED.      GBP  BBG000BD8278   CSRT  GB0000946276  0094627   
7        DIALIGHT      GBP  BBG000BLQNG4    DIA  GB0033057794  3305779   
8           GENUS      GBP  BBG000CTMKX7    GNS  GB0002074580  0207458   
9    GRESHAM TECH      GBP  BBG000BS2592    GHT  GB0008808825  0880882   

   marketcap  
0  4502.6800  
1 16349.6600  
2   185.2500  
3  3202.2000  
4  2579.9000  
5  1161.9600  
6   422.5600  
7   133.3900  
8  1455.7200  
9    64.3400

In [10]:
# upsert contituents to LUSID
fun.upsert_constituents(
    instrument_market_cap,
    holdings_effective_date,
    analyst_scope_code,
    reference_portfolio_code,
    api_factory,
)

Constituents Upserted


![Scopes](img/paper-referenceportfoliofilled.gif)

## 6) Allow our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

In [11]:
# Import our analyst's trades from a CSV file
analyst_transactions = import_data.fetch_client_transactions(
    "paper-transactions.csv", days_back=2
)
analyst_transactions.sort_values(by="transaction_id").head(n=10)

transaction_id       type                            portfolio  \
0   tid_124398219481        Buy  LSE_FTSE_techMarkFocus_transactions   
4   tid_223239424244   FundsOut  LSE_FTSE_techMarkFocus_transactions   
9   tid_239394829484        Buy  LSE_FTSE_techMarkFocus_transactions   
17  tid_324599699055   FundsOut  LSE_FTSE_techMarkFocus_transactions   
15  tid_334250655566        Buy  LSE_FTSE_techMarkFocus_transactions   
1   tid_339423984894   FundsOut  LSE_FTSE_techMarkFocus_transactions   
2   tid_339423984896   FundsOut  LSE_FTSE_techMarkFocus_transactions   
14  tid_342398598555   FundsOut  LSE_FTSE_techMarkFocus_transactions   
6   tid_342398989895  SellShort  LSE_FTSE_techMarkFocus_transactions   
20  tid_342453598349   FundsOut  LSE_FTSE_techMarkFocus_transactions   

   instrument_name instrument_uid                  transaction_date  \
0        AVEVA GRP   BBG000C21Y87  2021-03-21T16:20:20.081611+00:00   
4         GBP_Cash            GBP  2021-03-21T13:09:59.802555+00:00   
9         DIALIGHT   BBG000BLQNG4  2021-03-22T12:10:08.551547+00:00   
17        GBP_Cash            GBP  2021-03-22T15:02:47.499334+00:00   
15        MICROGEN   BBG000BDSS88  2021-03-22T09:49:11.843184+00:00   
1         GBP_Cash            GBP  2021-03-22T13:38:02.612234+00:00   
2         GBP_Cash            GBP  2021-03-21T14:45:00.664747+00:00   
14        GBP_Cash            GBP  2021-03-22T09:17:07.287098+00:00   
6    COMPUTACENTER   BBG000BN7CL9  2021-03-22T09:46:35.822369+00:00   
20        GBP_Cash            GBP  2021-03-22T16:03:48.004880+00:00   

                     settlement_date    units  transaction_price  \
0   2021-03-23T16:20:20.081611+00:00   265600            28.9400   
4   2021-03-23T13:09:59.802555+00:00   750000             0.0001   
9   2021-03-24T12:10:08.551547+00:00  1500000             4.1800   
17  2021-03-24T15:02:47.499334+00:00   234000             0.0020   
15  2021-03-24T09:49:11.843184+00:00   234000             3.9200   
1   2021-03-24T13:38:02.612234+00:00   265600             0.0058   
2   2021-03-23T14:45:00.664747+00:00   265600             0.0145   
14  2021-03-24T09:17:07.287098+00:00   345000             0.0110   
6   2021-03-24T09:46:35.822369+00:00   150000            10.5000   
20  2021-03-24T16:03:48.004880+00:00   543000             0.0068   

   transaction_currency   total_cost             strategy     description  
0                   GBP 7686464.0000   quantitativeSignal        Purchase  
4                   GBP      69.4500  fundamentalAnalysis  Brokerage Fees  
9                   GBP 6270000.0000   quantitativeSignal        Purchase  
17                  GBP     458.6400   quantitativeSignal      Stamp Duty  
15                  GBP  917280.0000   quantitativeSignal        Purchase  
1                   GBP    1537.2928   quantitativeSignal  Brokerage Fees  
2                   GBP    3843.2320   quantitativeSignal      Stamp Duty  
14                  GBP    3795.0000    technicalAnalysis      Stamp Duty  
6                   GBP 1575000.0000  fundamentalAnalysis        Purchase  
20                  GBP    3692.4000    technicalAnalysis      Stamp Duty

Custom properties can be defined in LUSID. In this example, a trading strategy is specified created as a **property**, this allows grouping of transactions for the purpose of measuring performance, valuation etc...

In [12]:
# Create a request to define our strategy property
strategy_property_key = fun.request_define_property(
    domain="Transaction",
    scope=analyst_scope_code,
    code="strategy",
    display_name="strategy",
    api_factory=api_factory,
)

Strategy Property Key: : Transaction/analyst-paper-396e-168b-6fe3-e8/strategy


In [13]:
# upsert all analyist transactions to LUSID
fun.upsert_trades(
    analyst_transactions,
    strategy_property_key,
    analyst_scope_code,
    transaction_portfolio_code,
    api_factory,
)

Transactions Successfully Upserted into Portfolio
Scope: analyst-paper-396e-168b-6fe3-e8
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Transactions Effective From: 2021-03-22 16:33:53.927595+00:00
Transactions Created On: 2021-03-23 15:01:21.602868+00:00



Our transaction portfolio is now populated with our analyst's paper trades.

## 7) Upload market data quotes to hold the latest prices of our tradeable instrument universe

In [14]:
# Import our instrument prices from a CSV file
instrument_prices = pd.read_csv("data/paper-analytics.csv")
# Pretty print our pricing
instrument_prices.sort_values(by="instrument_name").head(n=10)

instrument_name currency          figi  price_original  price_current  \
0       AVEVA GRP      GBP  BBG000C21Y87         27.8400        30.3178   
1        BAE SYS.      GBP  BBG000BD5TW4          5.1540         4.7829   
2   BATM ADVANCED      GBP  BBG000BFJD77          0.4590         0.4246   
3             BTG      GBP  BBG000BZZK79          8.2750         8.9701   
4          COBHAM      GBP  BBG000BS6810          1.0715         1.0222   
5   COMPUTACENTER      GBP  BBG000BN7CL9         10.6800        11.4596   
6    CONSORT MED.      GBP  BBG000BD8278          8.7000         9.3090   
7        DIALIGHT      GBP  BBG000BLQNG4          3.9200         3.5515   
8           GENUS      GBP  BBG000CTMKX7         22.4600        20.5958   
9    GRESHAM TECH      GBP  BBG000BS2592          0.9450         1.0008   

  ticker          isin    sedol  
0    AVV  GB00BBG9VN75  BBG9VN7  
1    BA.  GB0002634946   263494  
2    BVC  IL0010849045   911146  
3    BTG  GB0001001592   100159  
4    COB  GB00B07KD360  B07KD36  
5    CCC  GB00BV9FP302  BV9FP30  
6   CSRT  GB0000946276    94627  
7    DIA  GB0033057794  3305779  
8    GNS  GB0002074580   207458  
9    GHT  GB0008808825   880882

In [15]:
# Set our quote effective dates
quotes_effective_date = datetime.now(pytz.UTC) - timedelta(days=3)
today = datetime.now(pytz.UTC)

fun.create_instrument_quotes(
    quotes_effective_date, today, instrument_prices, analyst_scope_code, api_factory
)

![Scopes](img/paper-quotestore.gif)

## 8) Value each portfolio using our market data quotes

In [16]:
recipe_scope = "sandbox"
recipe_code = "market_value"

# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Equity.LusidInstrumentId.*",
                supplier="DataScope",
                data_scope=analyst_scope_code,
                quote_type="Price",
                field="Mid",
                quote_interval=quotes_effective_date.strftime("%Y-%m-%d"),
            )
        ],
        suppliers=models.MarketContextSuppliers(
            commodity="DataScope",
            credit="DataScope",
            equity="DataScope",
            fx="DataScope",
            rates="DataScope",
        ),
        options=models.MarketOptions(
            default_supplier="DataScope",
            default_instrument_code_type="LusidInstrumentId",
            default_scope=analyst_scope_code,
        ),
    ),
)

# Upsert the recipe
upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

# Create the valuation request
valuation_request = models.ValuationRequest(
    recipe_id=models.ResourceId(scope=recipe_scope, code=recipe_code),
    metrics=[
        models.AggregateSpec(key="Instrument/default/LusidInstrumentId", op="Value"),
        models.AggregateSpec(key="Instrument/default/Name", op="Value"),
        models.AggregateSpec(key="Holding/default/Units", op="Sum"),
        models.AggregateSpec(key="Holding/default/Cost", op="Sum"),
        models.AggregateSpec(key="Holding/default/PV", op="Sum"),
        models.AggregateSpec(key="Holding/default/Price", op="Sum"),
    ],
    group_by=["Instrument/default/LusidInstrumentId"],
    portfolio_entity_ids=[
        models.PortfolioEntityId(
            scope=analyst_scope_code, code=transaction_portfolio_code
        )
    ],
    valuation_schedule=models.ValuationSchedule(effective_at=today.isoformat()),
)

# Perform a valuation
valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request
)

df = pd.DataFrame(valuation.data)
df.loc["Total"] = df.sum(numeric_only=True)
df["return"] = (df["Sum(Holding/default/PV)"] - df["Sum(Holding/default/Cost)"]) / df[
    "Sum(Holding/default/Cost)"
]
df

Instrument/default/LusidInstrumentId Instrument/default/Name  \
0                                  CCY_GBP                     GBP   
1                            LUID_ECY4Y5DI             ULTRA ELEC.   
2                            LUID_42Q7OH5P               AVEVA GRP   
3                            LUID_0XICSR6Q           BATM ADVANCED   
4                            LUID_MH9ZRSXY               SAGE GRP.   
5                            LUID_YK15OEBB                   GENUS   
6                            LUID_0TTPI8L4           COMPUTACENTER   
7                            LUID_C6D5I9LV                MICROGEN   
8                            LUID_B5KMMX6J                DIALIGHT   
9                            LUID_QKL1C8AY                     SDL   
10                           LUID_VU0J71ZX             MICRO FOCUS   
11                           LUID_9011RJ64                XP POWER   
Total                                  NaN                     NaN   

       Sum(Holding/default/Units)  Sum(Holding/default/Cost)  \
0                    4364776.0000               4364776.0000   
1                     120000.0000               1582800.0000   
2                     265600.0000               7686464.0000   
3                     750000.0000                347250.0000   
4                     -98000.0000               -577220.0000   
5                     345000.0000               7590000.0000   
6                    -150000.0000              -1575000.0000   
7                     234000.0000                917280.0000   
8                    1500000.0000               6270000.0000   
9                      55000.0000                305250.0000   
10                   -543000.0000              -7384800.0000   
11                     98000.0000               2156000.0000   
Total                6941376.0000              21682800.0000   

       Sum(Holding/default/PV)  Sum(Holding/default/Price)  return  
0                 4364776.0000                     26.0000  0.0000  
1                 1603546.8000                     13.3629  0.0131  
2                 8052397.0560                     30.3178  0.0476  
3                  318431.2500                      0.4246 -0.0830  
4                 -643432.7200                      6.5656  0.1147  
5                 7105557.9000                     20.5958 -0.0638  
6                -1718946.0000                     11.4596  0.0914  
7                  843094.9800                      3.6030 -0.0809  
8                 5327280.0000                      3.5515 -0.1504  
9                  308154.0000                      5.6028  0.0095  
10               -7503461.7900                     13.8185  0.0161  
11                1912019.2000                     19.5104 -0.1132  
Total            19969416.6760                    154.8125 -0.0790

## 9) Compare performance across the analyst's reference and transaction portfolios

In [17]:
# create index portfolio
index_portfolio_code = "LSE_FTSE_techMarkFocus_index_DEMO"
prettyprint.heading("Index Portfolio Code", index_portfolio_code)

description = "Index transaction portfolio_DEMO"
fun.request_transaction_portfolio_creation(
    index_portfolio_code,
    portfolio_creation_date,
    analyst_scope_code,
    description,
    api_factory,
)

Index Portfolio Code: LSE_FTSE_techMarkFocus_index_DEMO
Portfolio Created
Scope: analyst-paper-396e-168b-6fe3-e8
Code: LSE_FTSE_techMarkFocus_index_DEMO
Portfolio Effective From: 2021-03-20 15:01:19.401250+00:00
Portfolio Created On: 2021-03-23 15:01:24.695905+00:00



We now have an index portfolio that we can use to hold our index and obtain our benchmark.

![Scopes](img/paper-indexportfolio.gif)

In [18]:
# setup an index portfolio
index_setup = fun.setup_index(
    analyst_scope_code, reference_portfolio_code, instrument_prices, api_factory
)

# Call LUSID to set all of our holdings to the initial index level
adjust_holdings_response = api_factory.build(
    lusid.api.TransactionPortfoliosApi
).set_holdings(
    scope=analyst_scope_code,
    code=index_portfolio_code,
    effective_at=portfolio_creation_date,
    adjust_holding_request=index_setup,
)

prettyprint.set_holdings_response(
    adjust_holdings_response, analyst_scope_code, index_portfolio_code
)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-396e-168b-6fe3-e8
Code: LSE_FTSE_techMarkFocus_index_DEMO
Holdings Effective From: 2021-03-20 15:01:19.401250+00:00
Holdings Created On: 2021-03-23 15:01:27.261324+00:00



We now have an index portfolio that we can value.

![Scopes](img/paper-indexportfoliofilled.gif)

In [19]:
# Create the valuation request
valuation_request = models.ValuationRequest(
    recipe_id=models.ResourceId(scope=recipe_scope, code=recipe_code),
    metrics=[
        models.AggregateSpec(key="Instrument/default/LusidInstrumentId", op="Value"),
        models.AggregateSpec(key="Instrument/default/Name", op="Value"),
        models.AggregateSpec(key="Holding/default/Units", op="Sum"),
        models.AggregateSpec(key="Holding/default/Cost", op="Sum"),
        models.AggregateSpec(key="Holding/default/PV", op="Sum"),
        models.AggregateSpec(key="Holding/default/Price", op="Sum"),
    ],
    group_by=["Instrument/default/LusidInstrumentId"],
    portfolio_entity_ids=[
        models.PortfolioEntityId(scope=analyst_scope_code, code=index_portfolio_code)
    ],
    valuation_schedule=models.ValuationSchedule(effective_at=today.isoformat()),
)

# Perform a valuation
valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request
)

df = pd.DataFrame(valuation.data)
df.loc["Total"] = df.sum(numeric_only=True)
df["return"] = (df["Sum(Holding/default/PV)"] - df["Sum(Holding/default/Cost)"]) / df[
    "Sum(Holding/default/Cost)"
]
df

Instrument/default/LusidInstrumentId Instrument/default/Name  \
0                            LUID_42Q7OH5P               AVEVA GRP   
1                            LUID_9UU2Z82D                BAE SYS.   
2                            LUID_0XICSR6Q           BATM ADVANCED   
3                            LUID_VA6SLBT9                     BTG   
4                            LUID_SRP4323T                  COBHAM   
5                            LUID_0TTPI8L4           COMPUTACENTER   
6                            LUID_SQNT32VV            CONSORT MED.   
7                            LUID_B5KMMX6J                DIALIGHT   
8                            LUID_YK15OEBB                   GENUS   
9                            LUID_0V8ILXJS            GRESHAM TECH   
10                           LUID_1HH59OBY                KCOM GRP   
11                           LUID_IXR7VNBO                 MEGGITT   
12                           LUID_VU0J71ZX             MICRO FOCUS   
13                           LUID_C6D5I9LV                MICROGEN   
14                           LUID_GHTW78PH                 NCC GRP   
15                           LUID_M60WMRDL          OXFORD BIOMED.   
16                           LUID_74E9M91P         OXFORD INSTRMNT   
17                           LUID_N8CFHDDZ           PADDY PWR BET   
18                           LUID_IKGEDDQZ                 QINETIQ   
19                           LUID_EM4TCO5Z                RENISHAW   
20                           LUID_AYMZI4LQ                 RICARDO   
21                           LUID_MH9ZRSXY               SAGE GRP.   
22                           LUID_QKL1C8AY                     SDL   
23                           LUID_ZNNWGYVC            SMITH&NEPHEW   
24                           LUID_0Y7S3SUP                SPECTRIS   
25                           LUID_BMOWRWJ3                 SPIRENT   
26                           LUID_M5JL0LXX              TRIAD GRP.   
27                           LUID_ECY4Y5DI             ULTRA ELEC.   
28                           LUID_87GBS9VC                 VECTURA   
29                           LUID_4P2OOVK4                    XAAR   
30                           LUID_9011RJ64                XP POWER   
Total                                  NaN                     NaN   

       Sum(Holding/default/Units)  Sum(Holding/default/Cost)  \
0                          2.0777                    57.8400   
1                         40.7525                   210.0400   
2                          5.1848                     2.3800   
3                          4.9713                    41.1400   
4                         30.9314                    33.1400   
5                          1.3977                    14.9300   
6                          0.6240                     5.4300   
7                          0.4371                     1.7100   
8                          0.8326                    18.7000   
9                          0.8747                     0.8300   
10                         6.6271                     4.6100   
11                        10.0275                    50.5800   
12                         5.3827                    79.6400   
13                         0.7919                     3.0600   
14                         3.5829                     6.4700   
15                         0.8445                     5.7800   
16                         0.7362                     6.3800   
17                         1.0262                    63.1100   
18                         7.4032                    22.2500   
19                         0.9505                    39.2400   
20                         0.7019                     4.2500   
21                        14.0411                    86.9700   
22                         1.1774                     6.5000   
23                        11.3173                   162.4600   
24                         1.5273                    36.3800   
25                         7.98